<a href="https://www.kaggle.com/code/buseozmntse/cnn-ile-trafik-aretleri-tan-mlama?scriptVersionId=264031870" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Projenin Amacı
Bu proje, trafik işaretlerinin görüntüler üzerinden Evrişimli Sinir Ağları (CNN) kullanılarak otomatik olarak sınıflandırılmasını hedeflemektedir. Trafik işaretlerinin doğru tanınması, otonom sürüş sistemleri ve sürücü destek teknolojileri için kritik bir adımdır.

Geliştirilen model, veri önişleme ve veri artırma (data augmentation) teknikleri ile eğitilerek 43 farklı trafik işareti sınıfını yüksek doğrulukla tanıyabilmektedir. Bu sayede, gerçek zamanlı uygulamalarda sürücülerin bilgilendirilmesi ve güvenli sürüş senaryolarının desteklenmesi amaçlanmaktadır.

# Veri Seti Hakkında
* Görseller 32x32 piksel boyutunda, RGB formatındadır.

* Veri seti üç parçaya ayrılmıştır:

  * Eğitim (train)

  * Doğrulama (validation)

  * Test (test)

* Toplamda 43 farklı trafik işareti sınıfı bulunmaktadır.

* Görseller, çeşitli ışık koşulları, açılar ve çevresel faktörleri içerecek şekilde hazırlanmıştır.


# Kullanılan Yöntemler
**Veri Önişleme**
 * Görsellerin normalize edilmesi (0–1 aralığına)
 * Etiketlerin one-hot encoding ile dönüştürülmesi
 * Train-validation-test ayrımı

**Data Augmentation**
 * Rotation (döndürme)
 * Width & Height Shift (kaydırma)
 * Zoom
 * Horizontal Flip

**Model Mimarisi = CNN**
 * Üç adet evrişim (Convolutional) katmanı:
   * 32, 64 ve 64 filtre
   * ReLu aktivasyon fonksiyonu
 * Her evrişim katmanından sonra MaxPooling katmanı
 * Dropout katmanları: 0.5
 * Tam bağlantılı (Dense) katman:
   * 64 nöron
   * Çıkış katmanı: 43 sınıf için Softmax aktivasyonu

**Model Değerlendirme**
* Accuracy & Loss grafikleri
* Confusion Matrix
* Classification Report

# Sonuç
* Modelin eğitim doğruluğu %93+, doğrulama doğruluğu ise %90 civarında.

* Bu sonuçlar trafik işareti sınıflandırması için oldukça başarılı.

* Ufak overfitting olmasına rağmen, dropout ve learning rate azaltma stratejisi ile kontrol altına alındı.

In [ ]:
# Dataset yüklendi
import os

data_path = "/kaggle/input/traffic-signs-preprocessed"

In [ ]:
# Gerekli kütüphaneler yüklendi
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import keras
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.utils import shuffle
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ----------- Veri Önişleme -----------
# Dataset klasöründeki dosyalar listelendi
print("Files in dataset folder:")
print(os.listdir(data_path))

In [ ]:
# Dataset içindeki pickle dosyalarının yolları tanımlandı
train_path = os.path.join(data_path, "train.pickle")
valid_path = os.path.join(data_path, "valid.pickle")
test_path  = os.path.join(data_path, "test.pickle")

In [ ]:
# Veriler yüklendi
# train
with open(train_path, 'rb') as f:
    train_data = pickle.load(f, encoding='latin1')
X_train, y_train = train_data['features'], train_data['labels']

# validation
with open(valid_path, 'rb') as f:
    valid_data = pickle.load(f, encoding='latin1')
X_valid, y_valid = valid_data['features'], valid_data['labels']

# test
with open(test_path, 'rb') as f:
    test_data = pickle.load(f, encoding='latin1')
X_test, y_test = test_data['features'], test_data['labels']

# Veri boyutları kontrol edildi
print("Train:", X_train.shape, y_train.shape)
print("Valid:", X_valid.shape, y_valid.shape)
print("Test:", X_test.shape, y_test.shape)

In [ ]:
# Eğitim setinden rastgele 5 örnek görselleştirildi
indices = np.random.choice(len(X_train), size=5, replace=False)
plt.figure(figsize=(10,2))
for i, idx in enumerate(indices):
    plt.subplot(1,5,i+1)
    plt.imshow(X_train[idx])
    plt.title(y_train[idx])
    plt.axis('off')
plt.show()

In [ ]:
# Normalization için veri tipi kontrolü
print(X_train.dtype)

In [ ]:
# Etiket isimleri getirildi
labels = pd.read_csv(data_path + "/label_names.csv")

# İlk satır getirildi
print("label_names.csv içindeki ilk 5 satır:")
print(labels.head())

# Sınıf sayısı kontrolü
print("\nSınıf sayısı:", labels.shape[0])

# Tüm sınıf kimlikleri isimleriyle birlikte yazdırıldı
print("\nTüm trafik işaret sınıfları:")
for i, name in enumerate(labels['SignName']):
    print(f"{i}: {name}")

In [ ]:
# Görseller normalize edildi (0 - 255 -> 0 - 1 aralığına)
X_train = X_train.astype('float32') / 255.0
X_valid = X_valid.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0

In [ ]:
# Label'lar One-Hot Encoding formatına çevrildi
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# ---------- Data Augmentation ---------------
datagen = ImageDataGenerator(
    rotation_range=15,       # resmi 15 dereceye kadar döndürme
    width_shift_range=0.1,   # yatay kaydırma
    height_shift_range=0.1,  # dikey kaydırma
    zoom_range=0.1,          # yakınlaştırma/uzaklaştırma
    horizontal_flip=True,     # yatay ayna simetrisi
)

# Veri artırma işlemi eğitim verisine uygulandı
datagen.fit(X_train)

In [ ]:
# Örnek olarak ilk görüntü getirildi
x = X_train[0].reshape((1,) + X_train[0].shape)  # (1, 32, 32, 3)

plt.figure(figsize=(8,8))
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.subplot(3, 3, i+1)
    plt.imshow(batch[0])
    plt.axis('off')
    i += 1
    if i == 9:  # 9 farklı augment edilmiş örnek göster
        break
plt.show()

In [ ]:
# --------- Modelin Eğitilmesi -----------
# CNN Modeli
# Model oluşturuldu
model = tf.keras.Sequential()

In [ ]:
# Katmanlar oluşturuldu 
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
# Model özeti 
model.summary()

In [ ]:
# Model derlendi
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Overfitting durumunu engellemek/dengelemek için
# Erken durdurma ile doğrulama kaybı uzun süre iyileşmezse eğitim durur
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Öğrenme oranını azaltma ile val_loss iyileşmezse öğrenme oranı küçültülür
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

# Model eğitildi
results = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# --------- Modelin Değerlendirilmesi ----------
plt.figure(figsize=(12,5))

# Loss grafiği
plt.subplot(1,2,1)
plt.plot(results.history['loss'], label='Train Loss')
plt.plot(results.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy grafiği
plt.subplot(1,2,2)
plt.plot(results.history['accuracy'], label='Train Accuracy')
plt.plot(results.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

**Modelin eğitim ve doğrulama başarımları birlikte artmış, aralarında belirgin bir fark oluşmamıştır. Validation loss uzun süre sabit kalarak aşırı öğrenmenin (overfitting) engellendiğini göstermektedir. EarlyStopping ve ReduceLROnPlateau callback’leri sayesinde eğitim süreci optimize edilmiştir. Sonuç olarak model yaklaşık %90 üzeri doğruluk ile genelleme yapabilmektedir.**

In [ ]:
# Test seti üzerinde değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Kaybı: {test_loss:.4f}")
print(f"Test Doğruluğu: {test_accuracy:.4f}")

# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification Report
print("\nSınıflandırma Raporu:")
print(classification_report(y_true, y_pred_classes))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Karmaşıklık Matrisi')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.show()

# Doğru ve Yanlış tahmin örnekleri
correct_indices = np.where(y_pred_classes == y_true)[0]
incorrect_indices = np.where(y_pred_classes != y_true)[0]

plt.figure(figsize=(12, 8))

# İlk 4 doğru tahmin gösterimi
for i, correct in enumerate(correct_indices[:4]):
    plt.subplot(2, 4, i+1)
    plt.imshow(X_test[correct])
    plt.title(f"Tahmin: {y_pred_classes[correct]}\nGerçek: {y_true[correct]}")
    plt.axis('off')

# İlk 4 yanlış tahmin gösterimi
for i, incorrect in enumerate(incorrect_indices[:4]):
    plt.subplot(2, 4, i+5)
    plt.imshow(X_test[incorrect])
    plt.title(f"Tahmin: {y_pred_classes[incorrect]}\nGerçek: {y_true[incorrect]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

**Sınıflandırma modeline göre model genel olarak %91 doğrulukla iyi performans gösteriyor ve çoğu sınıfı doğru sınıflandırıyor. Ancak bazı az örnekli sınıflarda (ör. 19, 20, 27, 37) ciddi yanlış sınıflandırmalar var, bu da veri dengesizliği ve sınıf sayısının az olmasından kaynaklanıyor.**

**Karmaşıklık Matrisi (Confusion Matrix) incelendiğinde, modelin performansının mükemmele yakın olduğu görülmüştür. Tahminlerin büyük çoğunluğu ana köşegen üzerinde yoğunlaşmaktadır ve bu, modelin genelleme yeteneğinin yüksek olduğunu ve trafik işaretlerini ayırt etme görevinde güvenilir ve tutarlı çalıştığını göstermektedir. Bu model yapısı, overfitting sorunu olmayan başarılı bir temel oluşturmaktadır.**

In [ ]:
# Heatmap Görselleştirme 
# Model Sequential'dı, Grad-CAM'de kullanılabilmesi için Functional API ile yeniden oluşturuldu
inputs = tf.keras.Input(shape=(32,32,3))
x = tf.keras.layers.Conv2D(32, (3,3), padding="same", activation="relu")(inputs)
x = tf.keras.layers.MaxPooling2D((2,2))(x)
x = tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="relu")(x)
x = tf.keras.layers.MaxPooling2D((2,2))(x)
x = tf.keras.layers.Conv2D(64, (3,3), padding="same", activation="relu", name="last_conv")(x)
x = tf.keras.layers.MaxPooling2D((2,2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Grad-CAM için Functional model oluşturuldu
last_conv_layer = model.get_layer("last_conv")
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[last_conv_layer.output, model.output]
)

In [ ]:
# Grad-CAM fonksiyonu
def get_gradcam(grad_model, image):
    img_array = np.expand_dims(image, axis=0)
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8
    
    return heatmap 

In [ ]:
#Görselleştirme yapıldı
indices = np.random.choice(len(X_test), size=5, replace=False)
plt.figure(figsize=(15,5))

for i, idx in enumerate(indices):
    image = X_test[idx]
    heatmap = get_gradcam(grad_model, image)  
    overlay = overlay_gradcam((image*255).astype('uint8'), heatmap)
    plt.subplot(1,5,i+1)
    plt.imshow(overlay)
    plt.title(f"Gerçek: {y_true[idx]}\nTahmin: {y_pred_classes[idx]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ----- Hiperparametre Optimizasyonu ------
from tensorflow.keras import regularizers

model = tf.keras.Sequential()
# Conv1
model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', 
                                 input_shape=(32,32,3),
                                 kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Dropout(0.4))

# Conv2
model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu',
                                 kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Dropout(0.4))

# Conv3
model.add(tf.keras.layers.Conv2D(96, (3,3), padding='same', activation='relu',
                                 kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Dropout(0.4))

# Dense
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(96, activation='relu',
                                kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

results = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
plt.figure(figsize=(12,5))

# Loss grafiği
plt.subplot(1,2,1)
plt.plot(results.history['loss'], label='Train Loss')
plt.plot(results.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy grafiği
plt.subplot(1,2,2)
plt.plot(results.history['accuracy'], label='Train Accuracy')
plt.plot(results.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Test seti üzerinde değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Kaybı: {test_loss:.4f}")
print(f"Test Doğruluğu: {test_accuracy:.4f}")

# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification Report
print("\nSınıflandırma Raporu:")
print(classification_report(y_true, y_pred_classes))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Karmaşıklık Matrisi')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.show()

# Doğru ve Yanlış tahmin örnekleri
correct_indices = np.where(y_pred_classes == y_true)[0]
incorrect_indices = np.where(y_pred_classes != y_true)[0]

plt.figure(figsize=(12, 8))

# İlk 4 doğru tahmin gösterimi
for i, correct in enumerate(correct_indices[:4]):
    plt.subplot(2, 4, i+1)
    plt.imshow(X_test[correct])
    plt.title(f"Tahmin: {y_pred_classes[correct]}\nGerçek: {y_true[correct]}")
    plt.axis('off')

# İlk 4 yanlış tahmin gösterimi
for i, incorrect in enumerate(incorrect_indices[:4]):
    plt.subplot(2, 4, i+5)
    plt.imshow(X_test[incorrect])
    plt.title(f"Tahmin: {y_pred_classes[incorrect]}\nGerçek: {y_true[incorrect]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

**Hiperparametre optimizasyonu kapsamında daha derin ve karmaşık bir CNN modeli denendi. Eğitim başarımı yüksek olmasına rağmen, doğrulama başarımı aynı seviyeye ulaşamadı. Train/validation kayıpları ve doğruluk grafikleri incelendiğinde overfitting gözlenmiştir. Bu deneme, modelin kapasitesinin veri seti için fazla olduğunu göstermektedir. Bu nedenle final model olarak daha dengeli sonuçlar veren ilk CNN modeli seçildi.**

**Sınıflandırma raporuna göre model genel olarak büyük sınıflarda iyi performans gösterse de, küçük sınıflarda ciddi hatalar yapıyor ve overfitting gösteriyor.**

**Karmaşıklık Matrisi, modelin hala yüksek bir genel doğruluğa sahip olduğunu gösterse de, overfitting'in somut etkilerini barındırmaktadır. Matristeki ana köşegenin dışında, özellikle görsel olarak benzer işaretlerin (örneğin hız limitleri) kesişim noktalarında belirgin hata kümeleri oluşmuştur. Bu, modelin eğitim verisini ezberlediği ve doğrulama/test verilerindeki ince farkları doğru şekilde genelleştiremediği anlamına gelir. Model, görevini çözmüş olsa da, tutarlılık ve güvenilirlik açısından ilk modelin gerisinde kalmaktadır.**